In [2]:
!pip install pyodbc polars grpcio-status jupysql --quiet
!apt update > /dev/null; apt install -y -q unixodbc > /dev/null


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip






In [8]:
from pyspark.sql import SparkSession

print("Getting Local Spark Session...")
spark = SparkSession.builder \
    .appName("Jupyter") \
    .getOrCreate()

# # Stop the local one
# SparkSession.builder.master("local[*]").getOrCreate().stop()
# print("Local Spark session stopped successfully.")

# # Create a remote one
# print("Starting remote Spark session...")
# spark = SparkSession.builder \
#     .appName("Jupyter") \
#     .remote("sc://localhost:15002") \
#     .getOrCreate()
# print("Created remote spark session successfully at sc://localhost:15002")

# spark

Getting Local Spark Session...


In [4]:
TABLE_NAME_REMOTE="Customers"
TABLE_NAME_LOCAL=f"local.db.{TABLE_NAME_REMOTE.lower()}"

In [ ]:
import os

# Import table structure from existing table
SQL_HOST=os.environ["SQL_HOST"]
SQL_USER=os.environ["SQL_USER"]
SQL_PASS=os.environ["SQL_PASS"]
SQL_PORT=1433
SQL_DATABASE=os.environ["SQL_DATABASE"]

# Construct JDBC URL
jdbc_url = f"jdbc:sqlserver://{SQL_HOST}:{SQL_PORT};databaseName={SQL_DATABASE}"

In [6]:
import pyodbc

try:
    # First create a temporary table
    spark.sql(f"""
    CREATE OR REPLACE TEMPORARY VIEW jdbc
    USING org.apache.spark.sql.jdbc
    OPTIONS (
        url '{jdbc_url}',
        dbtable '{TABLE_NAME_REMOTE}',
        user '{SQL_USER}',
        password '{SQL_PASS}',
        driver 'com.microsoft.sqlserver.jdbc.SQLServerDriver'
    )
    """)
    
    # Copy the full table from MSSQL
    spark.sql(f"""
        CREATE TABLE {TABLE_NAME_LOCAL}
        USING iceberg
        AS SELECT * FROM jdbc
    """)

    # Validate row counts
    source_count = spark.sql("SELECT COUNT(*) FROM jdbc").collect()[0][0]
    iceberg_count = spark.sql(f"SELECT COUNT(*) FROM {TABLE_NAME_LOCAL}").collect()[0][0]

    if source_count == iceberg_count:
        print(f"Full load successful! {source_count} rows loaded into Iceberg.")
    else:
        print(f"Row count mismatch: Source({source_count}) vs Iceberg({iceberg_count})")
except Exception as e:
    raise ValueError(f"Import failed: {str(e)}")


Full load successful! 11 rows loaded into Iceberg.


In [7]:
%sql SELECT * FROM $TABLE_NAME_LOCAL

25/04/03 13:15:30 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


CustomerID,FirstName,LastName,Email,CreatedDate,UpdatedDate
12,John,Smith,john.smith@example.com,2025-03-31 12:06:49.250000,2025-03-31 12:06:49.250000
13,Jane,Doe,jane.doe@example.com,2025-03-31 12:06:49.250000,2025-03-31 12:06:49.250000
14,Bob,Johnson,bob.johnson@example.com,2025-03-31 12:06:49.250000,2025-03-31 12:06:49.250000
15,Alice,Williams,alice.williams@example.com,2025-03-31 12:06:49.250000,2025-03-31 12:06:49.250000
16,Charlie,Brown,charlie.brown@example.com,2025-03-31 12:06:49.250000,2025-03-31 12:06:49.250000
17,Michael,Taylor,michael.taylor@example.com,2025-03-31 12:07:53.450000,2025-03-31 12:07:53.450000
18,Sarah,Johnson,sarah.johnson@example.com,2025-03-31 12:07:53.450000,2025-03-31 12:07:53.450000
19,David,Miller,david.miller@example.com,2025-03-31 12:07:53.450000,2025-03-31 12:07:53.450000
20,Emma,Wilson,emma.wilson@example.com,2025-03-31 12:07:53.450000,2025-03-31 12:07:53.450000
21,Thomas,Anderson,thomas.anderson@example.com,2025-03-31 12:07:53.450000,2025-03-31 12:07:53.450000


In [16]:
%%bash
# Install ODBC 18
if ! [[ "11 12" == *"$(grep VERSION_ID /etc/os-release | cut -d '"' -f 2 | cut -d '.' -f 1)"* ]];
then
    echo "Debian $(grep VERSION_ID /etc/os-release | cut -d '"' -f 2 | cut -d '.' -f 1) is not currently supported.";
    exit;
fi

# Download the package to configure the Microsoft repo
curl -sSL -O https://packages.microsoft.com/config/debian/$(grep VERSION_ID /etc/os-release | cut -d '"' -f 2 | cut -d '.' -f 1)/packages-microsoft-prod.deb
# Install the package
sudo dpkg -i packages-microsoft-prod.deb
# Delete the file
rm packages-microsoft-prod.deb

sudo apt-get update
sudo ACCEPT_EULA=Y apt-get install -y msodbcsql18
# optional: for bcp and sqlcmd
sudo ACCEPT_EULA=Y apt-get install -y mssql-tools18
echo 'export PATH="$PATH:/opt/mssql-tools18/bin"' >> ~/.bashrc
source ~/.bashrc
# optional: for unixODBC development headers
sudo apt-get install -y unixodbc-dev
# optional: kerberos library for debian-slim distributions
sudo apt-get install -y libgssapi-krb5-2

Selecting previously unselected package packages-microsoft-prod.
(Reading database ... 34662 files and directories currently installed.)
Preparing to unpack packages-microsoft-prod.deb ...
Unpacking packages-microsoft-prod (1.0-debian11.1) ...
Setting up packages-microsoft-prod (1.0-debian11.1) ...
Hit:1 http://deb.debian.org/debian bullseye InRelease
Hit:2 http://deb.debian.org/debian-security bullseye-security InRelease
Hit:3 http://deb.debian.org/debian bullseye-updates InRelease
Get:4 https://packages.microsoft.com/debian/11/prod bullseye InRelease [3650 B]
Get:5 https://packages.microsoft.com/debian/11/prod bullseye/main arm64 Packages [42.3 kB]
Get:6 https://packages.microsoft.com/debian/11/prod bullseye/main all Packages [1444 B]
Get:7 https://packages.microsoft.com/debian/11/prod bullseye/main armhf Packages [38.1 kB]
Get:8 https://packages.microsoft.com/debian/11/prod bullseye/main amd64 Packages [192 kB]
Fetched 278 kB in 0s (856 kB/s)
Reading package lists...
Reading package

debconf: delaying package configuration, since apt-utils is not installed


Fetched 688 kB in 0s (2969 kB/s)
Selecting previously unselected package msodbcsql18.
(Reading database ... 34670 files and directories currently installed.)
Preparing to unpack .../msodbcsql18_18.5.1.1-1_arm64.deb ...
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78.)
debconf: falling back to frontend: Readline
Unpacking msodbcsql18 (18.5.1.1-1) ...
Setting up msodbcsql18 (18.5.1.1-1) ...
odbcinst: Driver installed. Usage count increased to 1. 
    Target directory is /etc
Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  mssql-tools18
0 upgraded, 1 newly installed, 0 to remove and 27 not upgraded.
Need to get 208 kB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 https://packages.microsoft.com/debian/11/prod bullseye/mai

debconf: delaying package configuration, since apt-utils is not installed


Fetched 208 kB in 1s (146 kB/s)
Selecting previously unselected package mssql-tools18.
(Reading database ... 34689 files and directories currently installed.)
Preparing to unpack .../mssql-tools18_18.4.1.1-1_arm64.deb ...
Unpacking mssql-tools18 (18.4.1.1-1) ...
Setting up mssql-tools18 (18.4.1.1-1) ...
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78.)
debconf: falling back to frontend: Readline
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libodbc1 odbcinst odbcinst1debian2 unixodbc
Suggested packages:
  msodbcsql17 unixodbc-bin
The following NEW packages will be installed:
  unixodbc-dev
The following packages will be upgraded:
  libodbc1 odbcinst odbcinst1debian2 unixodbc
4 upgraded, 1 newly installed, 0 to remove and 23 not upgraded.
Need to 

debconf: delaying package configuration, since apt-utils is not installed


Fetched 752 kB in 0s (2280 kB/s)
(Reading database ... 34703 files and directories currently installed.)
Preparing to unpack .../libodbc1_2.3.11-1_arm64.deb ...
Unpacking libodbc1:arm64 (2.3.11-1) over (2.3.6-0.1+b1) ...
Preparing to unpack .../odbcinst1debian2_2.3.11-1_arm64.deb ...
Unpacking odbcinst1debian2:arm64 (2.3.11-1) over (2.3.6-0.1+b1) ...
Preparing to unpack .../odbcinst_2.3.11-1_arm64.deb ...
Unpacking odbcinst (2.3.11-1) over (2.3.6-0.1+b1) ...
Preparing to unpack .../unixodbc_2.3.11-1_arm64.deb ...
Unpacking unixodbc (2.3.11-1) over (2.3.6-0.1+b1) ...
Selecting previously unselected package unixodbc-dev.
Preparing to unpack .../unixodbc-dev_2.3.11-1_arm64.deb ...
Unpacking unixodbc-dev (2.3.11-1) ...
Setting up libodbc1:arm64 (2.3.11-1) ...
Setting up odbcinst (2.3.11-1) ...
Setting up odbcinst1debian2:arm64 (2.3.11-1) ...
Setting up unixodbc (2.3.11-1) ...
Setting up unixodbc-dev (2.3.11-1) ...
Processing triggers for libc-bin (2.31-13+deb11u11) ...
Reading package list

In [ ]:
# Fetch SQL Server CDC changes from Remote and merge them into the local Iceberg table
# we use pyodbc for this (to avoid temporary views)
import pyodbc
import sqlalchemy as sa
import polars as pl
from urllib.parse import quote_plus
from contextlib import contextmanager

LSN_DEFAULT = "0x00000000000000000000"

class SQLResource:
    def __init__(self, host, db, username, password, last_lsn=None):
        self.host = host
        self.db = db
        self.username = username
        self.password = password

        self.engine = sa.create_engine(
            self.get_connection_string(host, db, username, password)
        )

    def get_connection_string(self, host, db, username, password):
        """Construct the connection string for SQLAlchemy."""
        pass_escaped = quote_plus(password)
        user_escaped = quote_plus(username)
        driver_escaped = quote_plus("ODBC Driver 18 for SQL Server")
        return f"mssql+pyodbc://{user_escaped}:{pass_escaped}@{host}/{db}?driver={driver_escaped}"

    @contextmanager
    def get_connection(self):
        """Get a database connection using context manager for automatic cleanup."""
        connection = self.engine.connect()

        try:
            yield connection
        finally:
            connection.close()


    def get_primary_key_columns(self, table_name: str) -> list[str]:
        """Get the primary key columns for a CDC-enabled table."""
        with self.get_connection() as connection:
            instance = self.get_capture_instance_name("dbo", table_name)

            query = sa.text("""
            SELECT column_name FROM cdc.index_columns WHERE object_id = (
                SELECT object_id FROM cdc.change_tables WHERE capture_instance = :capture_instance_name
            )
            """)

            result = connection.execute(query, {"capture_instance_name": instance})
            primary_key_columns = [row[0] for row in result]
            return primary_key_columns

    def is_cdc_enabled_for_database(self):
        """Check if CDC is enabled for the database."""
        with self.get_connection() as connection:
            query = sa.text("""
                SELECT is_cdc_enabled 
                FROM sys.databases 
                WHERE name = :db_name
            """)
            result = connection.execute(
                query, {"db_name": self.db}
            ).scalar()
            return bool(result)

    def is_cdc_enabled_for_table(self, schema_name, table_name):
        """Check if CDC is enabled for the specified table."""
        with self.get_connection() as connection:
            capture_instance_name = self.get_capture_instance_name(
                schema_name, table_name
            )

            query = sa.text(f"""
                SELECT 1
                FROM cdc.change_tables
                WHERE capture_instance = '{capture_instance_name}'
            """)

            result = connection.execute(
                query, {"schema_name": schema_name, "table_name": table_name}
            ).scalar()

            return bool(result)

    def get_capture_instance_name(self, schema_name, table_name):
        """Get the CDC capture instance name for a table."""
        return f"dbo_{table_name}"

    def get_current_lsn(self):
        """Get the current  LSN from SQL Server using native function."""
        with self.get_connection() as connection:
            query = sa.text("SELECT sys.fn_cdc_get_max_lsn()")
            return connection.execute(query).scalar()

    def get_min_lsn(self, capture_instance=None):
        """Get the minimum available LSN for a capture instance."""
        with self.get_connection() as connection:
            query = sa.text("SELECT sys.fn_cdc_get_min_lsn(:capture_instance)")
            return connection.execute(
                query, {"capture_instance": capture_instance}
            ).scalar()

    def hex_string_to_lsn(self, lsn_hex):
        """Convert a hexadecimal LSN string to binary for SQL Server functions."""
        with self.get_connection() as connection:
            if not lsn_hex or not isinstance(lsn_hex, str):
                # Return minimum LSN if input is invalid
                query = sa.text("SELECT sys.fn_cdc_get_min_lsn(NULL)")
                return connection.execute(query).scalar()

            if not lsn_hex.startswith("0x"):
                lsn_hex = f"0x{lsn_hex}"

            query = sa.text("SELECT CAST(:lsn_hex AS BINARY(10))")
            result = connection.execute(query, {"lsn_hex": lsn_hex}).scalar()

            if result is None:
                query = sa.text("SELECT sys.fn_cdc_get_min_lsn(NULL)")
                return connection.execute(query).scalar()

            return result

    def lsn_to_hex_string(self, lsn_bytes):
        """Convert a binary LSN to a hex string format."""
        if lsn_bytes is None:
            return LSN_DEFAULT

        return f"0x{lsn_bytes.hex().upper()}"

    def get_primary_key_columns(self, table_name: str) -> list[str]:
        """Get the primary key columns for a CDC-enabled table."""
        with self.get_connection() as connection:
            instance = self.get_capture_instance_name("dbo", table_name)

            query = sa.text("""
            SELECT column_name FROM cdc.index_columns WHERE object_id = (
                SELECT object_id FROM cdc.change_tables WHERE capture_instance = :capture_instance_name
            )
            """)

            result = connection.execute(query, {"capture_instance_name": instance})
            primary_key_columns = [row[0] for row in result]
            return primary_key_columns

    def get_merge_predicate(self, table_name: str) -> str:
        """Uses the primary key columns to construct a predicate for merging.
        e.g., CustomerID and Email become: source.CustomerID = target.CustomerID AND source.Email = target.Email
        """
        primary_key_columns = self.get_primary_key_columns(table_name)
        if not primary_key_columns:
            raise ValueError(f"No primary key columns found for table {table_name}")

        # Construct the merge predicate
        merge_predicate = " AND ".join(
            [f"s.{col} = t.{col}" for col in primary_key_columns]
        )
        return merge_predicate

    def get_table_changes(
            self, table_name, last_lsn=None, schema_name="dbo", chunksize=10000
        ) -> tuple[pl.DataFrame, str]:
        """Get changes from a CDC-enabled table since the last LSN.
        Uses the native SQL Server CDC function fn_cdc_get_all_changes.

        Args:
            table_name (str): The name of the table to query.
            last_lsn (str, optional): The last processed LSN. If None, a full copy is performed.
            schema_name (str, optional): The schema name of the table. Defaults to 'dbo'.
            chunksize (int, optional): Number of rows to fetch per query. Defaults to 10000.

        Returns:
            tuple: A tuple containing the DataFrame of changes and the current LSN.
        """
        try:
            with self.get_connection() as connection:
                # Check if CDC is enabled for the database and table
                if not self.is_cdc_enabled_for_database():
                    raise ValueError(
                        f"CDC is not enabled for database {self.config.database.get_value()}"
                    )

                if not self.is_cdc_enabled_for_table(schema_name, table_name):
                    raise ValueError(
                        f"CDC not enabled for table {schema_name}.{table_name}"
                    )

                # Get the capture instance name
                capture_instance = self.get_capture_instance_name(
                    schema_name, table_name
                )
                if not capture_instance:
                    raise ValueError(
                        f"Could not find CDC capture instance for {schema_name}.{table_name}"
                    )

                # Get current maximum LSN
                current_lsn = self.get_current_lsn()
                current_lsn_hex = self.lsn_to_hex_string(current_lsn)
                
                # If no last_lsn provided, we should first take a first copy of the table
                if last_lsn is None or last_lsn == LSN_DEFAULT:
                    raise ValueError(
                        f"Initial copy required for table {schema_name}.{table_name}"
                    )

                # Convert LSN hex strings to binary
                from_lsn_hex = last_lsn
                to_lsn_hex = f"0x{current_lsn.hex()}"

                # Use the native CDC function with parameterized query
                # Process in chunks to avoid memory issues with large tables
                query = sa.text(f"""
                    DECLARE @from_lsn BINARY(10), @to_lsn BINARY(10)
                    SET @from_lsn = CONVERT(BINARY(10), :from_lsn, 1)
                    SET @to_lsn = CONVERT(BINARY(10), :to_lsn, 1)
        
                    SELECT * FROM cdc.fn_cdc_get_all_changes_{capture_instance}(
                        @from_lsn, @to_lsn, 'all'
                    )
                """)

                # Use chunksize to process large result sets in batches
                changes_df = pl.read_database(
                    query,
                    connection,
                    execute_options={
                        "parameters": {"from_lsn": from_lsn_hex, "to_lsn": to_lsn_hex}
                    },
                )

                # Convert binary LSN to hex string for storage
                current_lsn_hex = self.lsn_to_hex_string(current_lsn)

                return changes_df, current_lsn_hex

        except Exception as e:
            raise RuntimeError(
                f"Database error when getting CDC changes: {str(e)}"
            ) from e

# Connect to SQL Server
sql_resource = SQLResource(
    SQL_HOST,
    SQL_DATABASE,
    SQL_USER,
    SQL_PASS
)

last_lsn = "0x0000004400000D280005"

changes = sql_resource.get_table_changes(
    table_name=TABLE_NAME_REMOTE,
    last_lsn=last_lsn, # todo: fetch this each time and save into metadata
    schema_name="dbo"
)

# Make them available as temporary view
print(type(changes[0]))
changes_df = spark.createDataFrame(changes[0].to_pandas())
changes_df.createOrReplaceTempView("changes")

# print(changes)
print(spark.sql("SELECT * FROM changes").show())

<class 'polars.dataframe.frame.DataFrame'>
+--------------------+--------------------+------------+--------------+----------+---------+--------+--------------------+--------------------+--------------------+
|        __$start_lsn|           __$seqval|__$operation|__$update_mask|CustomerID|FirstName|LastName|               Email|         CreatedDate|         UpdatedDate|
+--------------------+--------------------+------------+--------------+----------+---------+--------+--------------------+--------------------+--------------------+
|[00 00 00 44 00 0...|[00 00 00 44 00 0...|           2|          [3F]|        23|  Michael|  Taylor|michael.taylor@ex...|2025-04-03 13:36:...|2025-04-03 13:36:...|
|[00 00 00 44 00 0...|[00 00 00 44 00 0...|           2|          [3F]|        24|    Sarah| Johnson|sarah.johnson@exa...|2025-04-03 13:36:...|2025-04-03 13:36:...|
|[00 00 00 44 00 0...|[00 00 00 44 00 0...|           2|          [3F]|        25|    David|  Miller|david.miller@exam...|2025-04-03

In [94]:
# Get the CDC Table Indices
merge_predicate = sql_resource.get_merge_predicate(
    table_name=TABLE_NAME_REMOTE
)

print(merge_predicate)

s.CustomerID = t.CustomerID


In [95]:
# Merge the CDC changes into the iceberg table as merge
# we work with the __$operation column to determine the type of operation, which can have values:
# - Deleted (__$operation = 1),
# - Inserted (__$operation = 2)
# - Updated Before (__$operation = 3)
# - Updated After (__$operation = 4)
# https://iceberg.apache.org/docs/1.5.0/spark-writes/#merge-into
print(f"Performing merge operation on '{TABLE_NAME_LOCAL}' with predicate '{merge_predicate}'...")
spark.sql(f"""
MERGE INTO {TABLE_NAME_LOCAL} AS t
USING (SELECT * FROM changes) AS s
ON {merge_predicate}
WHEN MATCHED AND s.`__$operation` = 1 THEN DELETE
WHEN MATCHED AND s.`__$operation` IN (2, 4) THEN UPDATE SET *
-- Anything we can't match, we insert
WHEN NOT MATCHED THEN INSERT *
""")

Performing merge operation on 'local.db.customers' with predicate 's.CustomerID = t.CustomerID'...


DataFrame[]

In [96]:
spark.sql(f"SELECT COUNT(*) FROM {TABLE_NAME_LOCAL}").show()

+--------+
|count(1)|
+--------+
|      13|
+--------+



In [98]:
spark.sql(f"SELECT * FROM {TABLE_NAME_LOCAL} ORDER BY UpdatedDate DESC").show()

+----------+---------+--------+--------------------+--------------------+--------------------+
|CustomerID|FirstName|LastName|               Email|         CreatedDate|         UpdatedDate|
+----------+---------+--------+--------------------+--------------------+--------------------+
|        29|  Michael|  Taylor|michael.taylor@ex...|2025-04-03 14:05:...|2025-04-03 14:05:...|
|        30|    Sarah| Johnson|sarah.johnson@exa...|2025-04-03 14:05:...|2025-04-03 14:05:...|
|        31|    David|  Miller|david.miller@exam...|2025-04-03 14:05:...|2025-04-03 14:05:...|
|        32|     Emma|  Wilson|emma.wilson@examp...|2025-04-03 14:05:...|2025-04-03 14:05:...|
|        33|   Thomas|Anderson|thomas.anderson@e...|2025-04-03 14:05:...|2025-04-03 14:05:...|
|        28|    Brave|   Sunny|brave.sunny@examp...|2025-04-03 13:55:...|2025-04-03 13:55:...|
|        23|  Michael|  Taylor|michael.taylor@ex...|2025-04-03 13:36:...|2025-04-03 13:36:...|
|        24|    Sarah| Johnson|sarah.johnson@exa..

In [ ]:
last_lsn = sql_resource.get_current_lsn()

In [99]:
# Clean the CDC table
# spark.sql(f"TRUNCATE TABLE {TABLE_NAME_LOCAL}")
sql_resource.cdc_clear(TABLE_NAME_REMOTE)

CDC data cleared for Customers.
